In [ ]:
__autor__ = "Felipe Alejandro Pizarro Marquez"
__credits__ = ["Felipe Alejandro Pizarro Marquez",
               "Jose David Marroquin Toledo", ]
__email__ = ["fpizarro92@live.cl", "jose@marroquin.cl", ]
__status__ = "Development"

## Using openMVG and openMVG to Build a 3D Model

This module allows to **reconstruct a 3D Model from a image set of an object captured with a camera from diferent angles**. For that, in first place, with [openMVG](https://github.com/openMVG/openMVG) (by [pmoulon](https://github.com/pmoulon)) it is generated a **dense cloud point** and then, with [openMVS](https://github.com/cdcseacave/openMVS) (by [cdcseacave](https://github.com/cdcseacave)), it is generated the **fully textured 3D mesh**.

[MvgMvs_Pipeline.py](../scripts/MvgMvs_Pipeline.py) [1] by [FlachyJoe](byhttps://github.com/FlachyJoe) does easy the reconstruction process with openMVG and openMVS. [sumenage](https://github.com/sumenage) did it compatible with Pyhton 3 and we had to edited and added it some code lines.

[1] FlachyJoe. (2016). *[Enhancement] OpenMVG + OpenMVS python pipeline [Msg 1]*. Message posted to  https://github.com/cdcseacave/openMVS/issues/115

In [ ]:
import os
import subprocess
import shutil
import PIL.Image
import blendjupyter as blendj

In [ ]:
def create_3d_model(**kwargs):
    """Returns the route of the output folder generated that contains
    the files and folders generated with during the reconstruction
    process of the 3D model."""
    reslevel = kwargs.pop('reslevel', 1)
    scale = kwargs.pop('scale', 1)  # Image scaling (0, 1].
    bw = kwargs.pop('bw', False)  # For grayscale convertion.
    sample = kwargs.pop('sample', False)     
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'super-scanner-software-s3'))   
    inpath = kwargs.pop('inpath',os.path.join(os.path.expanduser('~'),
                                              's3-out','scanner',
                                              'blend-phg-set-0001'))    
    outdirname = kwargs.pop('outdirname', 
                            os.path.join(inpath,'phg-model-0001'))
    exif = kwargs.pop('exif',
                      os.path.join(s3path,
                                   'img',
                                   'Photo 25-09-16 11 11 00.jpg'))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(create_3d_model.__name__, list(kwargs.keys())[-1]))
    if sample is True:  # Use ArtemisStatue sample set generated with
                        # Super Scanner Software (S3).
        inpath = os.path.join(s3path, 'img','photog',
                              'ArtemisStatue-S3-phg-set')
        if not os.path.exists(inpath):
            print('No such directory:', os.path.basename(inpath))
            print('Did you uncompress ' +  os.path.basename(inpath) + '.zip?')
            return None
        outdirname = os.path.join(os.path.expanduser('~'), 's3-out',
                                  'scanner',
                                  'ArtemisStatue-phg-model-0001')
    outpath = blendj.find_out_dir(dirname=outdirname,
                                  parentdir='scanner')
    if scale < 1:
        img_path = os.path.join(outpath, 'scaled-set')
        man_input_img(inpath, img_path, gray=bw, scalefactor = scale)
    elif bw is True:
        img_path = os.path.join(outpath, 'bw-set')        
        man_input_img(inpath, img_path, gray=bw)
    else:
        img_path = inpath
    l_ls = !ls $img_path
    for name in l_ls:  # First, give Exif metadata to the source
                       # images generated by Blender.
        path_file = os.path.join(img_path, name)
        if '.jpg' in name.lower() or '.jpeg' in name.lower():
            blendj.copy_exif(path_file, exif)
    print('Input image folder: %s' %inpath)
    print('Output path with the reconstruction: %s'  %outpath)
    mvg_mvs_script_path = os.path.join(s3path, 'scripts',
                                       'MvgMvs_Pipeline.py')
    # Call the Python script pipeline to use openMVG and openMVS with
    # custom parameters.
    print('Execute:', mvg_mvs_script_path)
    print('See the command-line from where you launched Jupyter.')
    subprocess.call(['python3', mvg_mvs_script_path, img_path,
                     outpath,
                     '--1', 'm', 'AKAZE_FLOAT', 'p', 'HIGH',
                     '--2', 'r', '0.7',
                     '--8', 'resolution-level', str(reslevel),
                     '--10', 'resolution-level', str(reslevel),
                     '--11', 'resolution-level', str(reslevel)])
    print('End of the execution of', os.path.basename(mvg_mvs_script_path))
    return outpath

In [ ]:
def open_3d_model(folder, **kwargs):
    """Opens the reconstructed model in folder (str) with an editor.
    """
    # mesh (str) can be 'TEXTURED' or 'MODEL'.
    mesh = kwargs.pop('mesh', 'TEXTURED')
    # Check if the selected editor exists.
    editor = kwargs.pop('editor', 'meshlab')
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(open_3d_model.__name__, list(kwargs.keys())[-1]))
    editor_switcher = {'meshlab': 'meshlab'}
    editor = editor_switcher.get(editor.lower(), False)
    assert editor != False, 'Incorrect editor.'
    # Check the mesh type.
    mesh_switcher = {'textured': 'textured', 'model': 'model'}
    mesh = mesh_switcher.get(mesh.lower(), False)
    assert mesh != False, 'Incorrect type of mesh.'
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'
                        .format(open_3d_model.__name__, list(kwargs.keys())[-1]))
    if mesh.lower() == 'model':
        obj = os.path.join(folder, 'mvs',
                           'scene_dense_mesh_refine.ply')
    elif mesh.lower() == 'textured':
        obj = os.path.join(folder, 'mvs',
                           'scene_dense_mesh_refine_texture.ply')
    assert os.path.isfile(obj) == True, os.path.basename(obj) + ' does not exist.'
    if editor is 'meshlab':  # Use meshlab to view or edit the mesh.
        subprocess.Popen([editor, obj])

In [ ]:
def man_input_img(input_path, output_path, **kwargs):
    """Manages the input image path."""
    scalefactor = kwargs.pop('scalefactor', 1)
    gray = kwargs.pop('gray', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(man_input_img.__name__, list(kwargs.keys())[-1]))
    exts = ('.jpg', '.JPG')
    os.makedirs(output_path) 
    for file in os.listdir(input_path):
        if os.path.isfile(os.path.join(input_path, file)):
            if any(file.lower().endswith(ext) for ext in exts):
                shutil.copy2(os.path.join(input_path, file),
                             os.path.join(output_path, file))
                if scalefactor is not 1 or gray is True:
                    scale_img_PIL(os.path.join(output_path,file),
                                  scalefactor, bw=gray)
                    blendj.copy_exif(os.path.join(output_path, file),
                                     os.path.join(input_path, file))

In [ ]:
def scale_img_PIL(img_path, scale_factor, **kwargs):
    """Using PIL.Image Python module, it scales scale_factor (float)
    times the images within of img_path (str). Furthermore, it can
    convert them to grayscale."""    
    bw = kwargs.pop('bw', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(scale_img_PIL.__name__, list(kwargs.keys())[-1]))
    img = open(img_path, "rb")
    if bw:
        img = PIL.Image.open(img).convert('L')
    else:
        img = PIL.Image.open(img)
    if scale_factor < 1:
        img = img.resize([int(scale_factor * s) for s in img.size],
                         PIL.Image.ANTIALIAS)
    img = img.convert('RGB')
    img.save(img_path) 
    img.close()